In [ ]:
!pip install caer canaro

# Simpson Family Deep Learning Model

In [ ]:
import os
import caer
import canaro
import numpy as np
import cv2 as cv
import gc

In [ ]:
IMG_SIZE = (80,80)
channel = 1
char_path = R'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [ ]:
char_dict = {}
for folder in os.listdir(char_path):
    char_dict[folder] = len (os.listdir(os.path.join(char_path, folder)))
# Sort the Dictionary in Descending order
char_dict = caer.sort_dict(char_dict, descending = True)


In [ ]:
characters = []
idx = 0
for character in char_dict:
    idx +=1
    characters.append(character[0])
    if idx >= 10:
        break
characters

In [ ]:
# Create The training Data
train = caer.preprocess_from_dir (char_path, characters, channels = channel, IMG_SIZE = IMG_SIZE, isShuffle = True)


In [ ]:
# See how many images there are in the training set
len(train)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (30,30))
plt.imshow(train[0][0], cmap = 'gray')
plt.show()

In [ ]:
# Seperate training into features and labels
featuresSet, labels = caer.sep_train(train, IMG_SIZE = IMG_SIZE)

In [ ]:
from tensorflow.keras.utils import to_categorical
# Normalize the featureSet
# to be the range of (0,1)
featuresSet = caer.normalize(featuresSet)
labels = to_categorical(labels, len(characters))

In [ ]:
# Training the data and validation data
x_train, x_val, y_train, y_val = caer.train_val_split(featuresSet, labels, val_ratio = 0.2)

In [ ]:
# remove variables that are not used
del train
del featuresSet
del labels
gc.collect()

In [ ]:
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
# Because of the library update, we need to convert 
# python array to numpy array
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)
# Image data Generator
data_generated = canaro.generators.imageDataGenerator()
train_generated = data_generated.flow(x_train, y_train, batch_size = BATCH_SIZE)


In [ ]:
# Create the model
model = canaro.models.createSimpsonsModel (IMG_SIZE  = IMG_SIZE, channels = channel, output_dim = len(characters),
                                          loss = 'binary_crossentropy', decay = 1e-6, learning_rate = 0.001,
                                          momentum = 0.9, nesterov  = True)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
# Create Callback Lists
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [ ]:
# Train the model
training = model.fit (train_generated, steps_per_epoch = len(x_train)//BATCH_SIZE, epochs = EPOCHS,validation_data = 
                     (x_val, y_val), validation_steps = len(y_val) // BATCH_SIZE,
                     callbacks =  callbacks_list)

In [ ]:
# Test the model
test_path = R'../input/the-simpsons-characters-dataset/simpsons_dataset/bart_simpson/pic_0019.jpg'
img = cv.imread(test_path)
# plt.imshow(img, cmap = 'gray')
# plt.show()
def prepare (img):
    # Prepare the img to be the same size of the model accepted
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, IMG_SIZE)
    img = caer.reshape(img, IMG_SIZE, 1)
    return img


In [ ]:
predictions = model.predict(prepare(img))

In [ ]:
predictions